# Multivariate Chebyshev

In this notebook we analyse multivariate Chebyshev interpolation. For reference, see
[L. N. Trefethen *Approximation Theory and Approximation Practice* (2013)](https://people.maths.ox.ac.uk/trefethen/ATAP/ATAPfirst6chapters.pdf)
and
[M.Gaß, K. Glau, M. Mahlstedt, M. Mair, *Chebyshev Interpolation for Parametric Option Pricing* (2016)](https://arxiv.org/pdf/1505.04648.pdf).

In [ ]:
import numpy as np
from numpy.polynomial import Chebyshev
import pandas as pd
from scipy.stats import norm

import matplotlib.pyplot as plt

## One-dimensional Chebyshev Interpolation

We consider a function $f:[-1, 1]\rightarrow \mathbb{R}$. The function $f$ is approximated by a sum of polynomials of the form
$$
  f(x) \approx \sum_{k=0}^n c_k T_k(x).
$$
Here,
  - $n$ is the order of polynomial approximation,
  - $T_k$ is the $k$-th element of the Chebyshev series (basis function, polynomial of degree $k$), and
  - $c_k$ are the coefficients for approximation (obtained by interpolation through Chebyshev nodes).

### Chebychev via First Kind

We illustrate the first four Chebyshev polynomials using numpy.

In [ ]:
T0 = Chebyshev([1])
T1 = Chebyshev([0, 1])
T2 = Chebyshev([0, 0, 1])
T3 = Chebyshev([0, 0, 0, 1])

x = np.linspace(-1.0, 1.0, 101)
fig, axs = plt.subplots(2,2)
fig.suptitle('Chebyshev polynomials')
axs[0,0].plot(x, T0(x))
axs[0,1].plot(x, T1(x))
axs[1,0].plot(x, T2(x))
axs[1,1].plot(x, T3(x))
plt.show()

For a given function $f$ we can also calculate the coefficients $c_k$ ($k=0,\ldots, n$).

We illustrate coefficient calculation by approximating $f(x)=e^x$.

In [ ]:
poly = Chebyshev.interpolate(np.exp, 3)
poly

We can extract the coefficients from our interpolation polynomial.

In [ ]:
poly.coef

And we can also manually construct an approximating Chebyshev polynomial. 

In [ ]:
poly2 = Chebyshev(poly.coef)
poly2

Both polynomials are equivalent. And the difference is *zero*.

In [ ]:
poly - poly2

Finally, we check the approximation of the test function.

In [ ]:
fig, axs = plt.subplots(1,2)
axs[0].plot(x, poly(x))
axs[0].plot(x, np.exp(x))
axs[1].plot(x, np.exp(x)-poly(x))
plt.tight_layout()
plt.show()

Numpy uses Chebychev nodes of the fist kind.

In [ ]:
cheb_nodes_1st = np.polynomial.chebyshev.chebpts1(3+1)
print(cheb_nodes_1st)
print(np.exp(cheb_nodes_1st)-poly(cheb_nodes_1st))

### Chebychev via Second Kind

Following Glau et.al. we also check interpolation using Chebychev nodes of the second kind.

In [ ]:
n = 3  # degree of polynomial/interpolation

cheb_nodes_2nd = np.cos(np.pi * np.linspace(0, 1, n+1))
print(cheb_nodes_2nd)
print(np.flip(np.polynomial.chebyshev.chebpts2(n+1)))

We need to evaluate the target function at the nodes.

In [ ]:
f_nodes = np.exp(cheb_nodes_2nd)
f_nodes

Now, we can calculate the coefficients of the Chebychev polynomial.

In [ ]:
f_nodes_adj = f_nodes.copy()
f_nodes_adj[0] *= 0.5
f_nodes_adj[-1] *= 0.5
#
idxs = np.arange(n+1)
fac = 2**((0 < idxs) & (idxs < n))
cheb_coeff = np.dot(np.cos((np.pi * idxs.reshape(-1,1)) * np.linspace(0, 1, n+1)), f_nodes_adj / n) * fac
print(cheb_coeff)
poly_3 = Chebyshev(cheb_coeff)
poly_3

It turns out that this polynomial slightly differs from numpy's coefficients for interpolation.

Again, we check the interpolation and difference to the target function.

In [ ]:
fig, axs = plt.subplots(1,2)
axs[0].plot(x, poly_3(x))
axs[0].plot(x, np.exp(x))
axs[1].plot(x, np.exp(x)-poly_3(x))
plt.tight_layout()
plt.show()

In [ ]:
print(np.exp(cheb_nodes_2nd)-poly_3(cheb_nodes_2nd))

As expected we match the target function at the Chebychev nodes of second kind. However, the maximum approximation error slightly increased compared to numpy's implementation. 

## Chebychev Tensors

Now, we analyse multivariate interpolation. The implementation also follows Glau et.al.

In [ ]:
def cartesian_product(*arrays):
    """
    Calculate the cartesian product of a list of input arrays.

    See https://stackoverflow.com/questions/11144513/cartesian-product-of-x-and-y-array-points-into-single-array-of-2d-points
    """
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

class ChebychevTensor:
    """
    A multi-variate interpolation using Chebychev tensors.
    """

    def __init__(self, func, degrees, a = -1.0, b = 1.0):
        """
        Initialise a d-dimensional ChebychevTensor.

        Inputs:

        func    ... A function taking an array of length d as input.
        degrees ... A list of length d containing integers. Elements
                    represent degree of polynomial for corresponding
                    dimension i (i=1,..,d).
        a, b    ... Lower/upper boundary for domain; optional,
                    scalar or d-dimensional array.
        """
        self.func = func
        self.degrees = degrees
        self.a = a
        self.b = b
        #
        idx_list = [ np.arange(n+1) for n in self.degrees]
        points = self.chebychev_points(self.degrees)
        #
        self.multi_idx = cartesian_product(*idx_list)
        self.multi_points = cartesian_product(*points)
        # we need to scale the points from [-1,1] to [a,b]
        if not isinstance(a, float):
            self.a = self.a.reshape((1,-1))  # allow broadcast for calibration
        if not isinstance(b, float):
            self.b = self.b.reshape((1,-1))  # allow broadcast for calibration
        self.multi_points = self.a + 0.5 * (self.multi_points + 1) * (self.b - self.a)
        if not isinstance(a, float):
            self.a.shape = (-1,)  # allow broadcast for valuation
        if not isinstance(b, float):
            self.b.shape = (-1,)  # allow broadcast for valuation
        #
        self.values = np.apply_along_axis(self.func, 1, self.multi_points)
        #
        self.inner_node_factor = np.sum(
            (0<self.multi_idx) & 
            (self.multi_idx<np.array(self.degrees)), 
            axis = 1
        )               
        self.values_adj = self.values * 0.5**(len(self.degrees) - self.inner_node_factor)
        #
        self.multi_coeff = np.zeros(self.multi_idx.shape[0])
        for idx, j in enumerate(self.multi_idx):
            # calculate c_j for multi-index j, see eq. (2.9)
            cos_list = [
                np.array([ np.cos(j[i] * np.pi * k / self.degrees[i]) for k in range(self.degrees[i]+1) ])
                for i in range(len(self.degrees))
            ]
            cos_full = cartesian_product(*cos_list)
            self.multi_coeff[idx] = np.dot(self.values_adj, np.prod(cos_full, axis=1))
        self.multi_coeff *= 2**self.inner_node_factor / np.prod(self.degrees)
        #
        self.T = [ np.polynomial.Chebyshev.basis(j) for j in range(np.max(self.degrees) + 1) ]

    @staticmethod
    def chebychev_points(degrees):
        """
        Calculate the Chebychev points used for interpolation.

        These points represent the extrema (!) of Chebychev
        polynomials of first kind.
        """
        return [ np.cos(np.pi * np.linspace(0, 1, n+1)) for n in degrees]

    def chebychev_basis(self, x):
        """
        Evaluate all basis function tensors at x.

        This assumes x in [-1,1].
        """
        assert x.shape[0] == len(self.degrees)
        T_list = [
            np.array([ self.T[j](x[i]) for j in range(self.degrees[i]+1) ])
            for i in range(len(self.degrees))
        ]
        T_full = cartesian_product(*T_list)
        return np.prod(T_full, axis=1)

    def __call__(self, y):
        """
        Evaluate interpolation for d-dimensional input y.
        """
        x = 2 * (y-self.a) / (self.b-self.a) - 1
        return np.dot(self.multi_coeff, self.chebychev_basis(x))

We test the multi-variate interpolation using (normalised) Black formula.

In [ ]:
def BlackOverK(x):
    moneyness, stdDev, callOrPut = x
    d1 = np.log(moneyness) / stdDev + stdDev / 2.0
    d2 = d1 - stdDev
    return callOrPut * (moneyness*norm.cdf(callOrPut*d1)-norm.cdf(callOrPut*d2))

In [ ]:
a = np.array([ 0.5, 0.01, -1.0 ])
b = np.array([ 2.0, 0.50, +1.0 ])
degrees = [ 10, 10, 10 ]

T = ChebychevTensor(BlackOverK, degrees, a, b)

In [ ]:
callOrPut = -1.0
moneyness = np.linspace(0.5, 2.0, 101)

fig, axs = plt.subplots(2,2, sharex=True, sharey=True)

for stDev, ax in zip([ 0.01, 0.10, 0.25, 0.50 ], axs.reshape(-1)):
    v = lambda x : np.array([x, stDev, callOrPut])
    ax.plot(moneyness, [ BlackOverK(v(x)) for x in moneyness], 'r', label='Black')
    ax.plot(moneyness, [ T(v(x)) for x in moneyness], 'b', label='Chebychev')
    ax.set_title('stDev = %.2f' % stDev)
axs.reshape(-1)[0].legend()
axs[1,0].set_xlabel('moneyness')
axs[1,1].set_xlabel('moneyness')
axs[0,0].set_ylabel('forward price')
axs[1,0].set_ylabel('forward price')
fig.set_figheight(8)
fig.set_figwidth(12)
fig.suptitle('Call/Put = %.1f' % callOrPut)
plt.tight_layout()
plt.show()

For a more systematic view we sample random points from the interpolation domain and check approximation accuracy for that sample.

In [ ]:
np.random.seed(42)
base2 = 13
X = a.reshape(1,-1) + np.random.uniform(size=(2**base2, 3)) * (b-a).reshape(1,-1)

Reference values are calculated using Black formula.

In [ ]:
V_ref = np.apply_along_axis(BlackOverK, 1, X)

Model values are calculated using Chebychev tensors.

In [ ]:
V_mdl = np.apply_along_axis(T, 1, X)

We compare standard statistics of the difference between (proxy) model and target function. 

In [ ]:
pd.Series((V_mdl - V_ref), name='Model - Target').describe()